# ДПО ВШЭ

## Современный анализ данных, глубокое обучение и приложения

## Задание на тему: "Рекомендательные системы"

В этом задании будем практиковаться в реализации рекомендательных систем.

Воспользуемся небольшим датасетом с Kaggle: [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

In [ ]:
import numpy as np
import scipy
import pandas as pd
import math

%matplotlib inline
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

## Часть 0. Загрузка данных

Загрузим [Deskdrop dataset](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop), включающийся в себе логи за 1 год платформы, где пользователи читают статьи.

Данные включают в себя 2 файла:  
- **shared_articles.csv**
- **users_interactions.csv**

Как можно догадаться, в одном описания самих статей (нам понадобятся в контентных моделях), а в другом логи пользователей.

#### shared_articles.csv

Так как в файле перечислены даже удалённые статьи, то мы их сразу удалим (на самом деле они могли бы быть нам полезны для подсчёта различных величин, хоть мы и не можем их рекомендовать).

In [ ]:
articles_df = pd.read_csv('data/shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(5)

#### users_interactions.csv

В колонке eventType описаны действия, которые могли совершать пользователи над статьёй:  
- VIEW
- LIKE
- COMMENT CREATED
- FOLLOW
- BOOKMARK

In [ ]:
interactions_df = pd.read_csv('data/users_interactions.csv')
interactions_df.head(10)

In [ ]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

### Предобработка данных

В логах встречаются различные действия пользователей. Однако мы хотим работать лишь с одной величиной, характеризующей всё взаимодействие пользователя со статьёй. Предлагается задать действиям следующие веса:

In [ ]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

Посчитайте числовую величину "оценки" пользователем статьи с указанными выше весами.

In [ ]:
interactions_df['eventStrength'] = # your code

Ремендательные системы подвержены проблеме холодного старта. В рамках данного задания предлагается работать только с теми пользователями, которые взаимодействовали хотя бы с 5 материалом.

Оставьте только таких пользователей. Их должно остаться 1140.

In [ ]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())
print('# users:', len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
print('# users with at least 5 interactions:',len(users_with_enough_interactions_df))

Оставьте только те взаимодействия, которые касаются только отфильтрованных пользователей.

In [ ]:
interactions_from_selected_users_df = # your code

In [ ]:
print('# interactions before:', interactions_df.shape)
print('# interactions after:', interactions_from_selected_users_df.shape)

В данной постановке каждый пользователей мог взаимодействовать с каждой статьёй более 1 раза (как минимум совершая различные действия). Предлагается "схлопнуть" все действия в одно взаимодействие с весом, равным сумме весов. 

Однако полученное число будет в том числе тем больше, чем больше действий произвёл человек. Чтобы уменьшить разброс предлагается взять логарифм от полученного числа (можно придумыват другие веса действиям и по-другому обрабатывать значения).

Также сохраним последнее значение времени взаимодействия для разделениея выборки на обучение и контроль.

In [ ]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(10)

Разобьём выборку на обучение и контроль по времени.

In [ ]:
split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказанями в виде списков.

In [ ]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), 'true_test'] = [
    list() for x in range(len(interactions.loc[pd.isnull(interactions.true_test), 'true_test']))]

interactions.head(1)

## Часть 1: Baseline (модель по популярности)

Самой простой моделью рекомендаций (при этом достаточно сильной!) является модель, которая рекомендует наиболее популярные предметы. 

Предлагается реализовать её. Давайте считать, что рекомендуем мы по 10 материалов (такое ограничение на размер блока на сайте).

Посчитайте популярность каждой статьи, как сумму всех "оценок" взаимодействий с ней. Отсортируйте материалы по их популярности.

In [ ]:
popular_content = # your code

Теперь необходимо сделать предсказания для каждого пользователя. Не забывайте, что надо рекомендовать то, что пользователь ещё не читал (для этого нужно проверить, что материал не встречался в true_train).

In [ ]:
top_k = 10

interactions['prediction_popular'] = # your code

Настало время оценить качество. Посчитайте precision@10 для каждого пользователя (доля угаданных рекомендаций). Усредните по всем пользователям. Везде далее будем считать эту же метрику.

In [ ]:
# your code

## Часть 2. Коллаборативная фильтрация.

Перейдём к более сложному механизму рекомендаций, а именно коллаборативной фильтрации. Суть коллаборативной фильтрации в том, что учитывается схожесть пользователей и товаров между собой, а не факторы, которые их описывают. 

__Предлагается на выбор реализовать один из двух подходов__: memory-based или модель со скрытыми переменные.

Для начала для удобства составим матрицу "оценок" пользователей. Нули будут обозначать отсутствие взаимодействия.

In [ ]:
ratings = pd.pivot_table(
    interactions_train_df,
    values='eventStrength',
    index='personId',
    columns='contentId').fillna(0)

### Memory-based

Посчитайте схожести пользователей с помощью корреляции Пирсона. Для каждой пары учитываем только ненулевые значения.

Для скорости работы лучше переходить от pandas к numpy.

In [ ]:
ratings_m = ratings.as_matrix()

In [ ]:
similarity_users = np.zeros((len(ratings_m), len(ratings_m)))

for i in tqdm_notebook(range(len(ratings_m)-1)):
    for j in range(i+1, len(ratings_m)):
        
        # nonzero elements of two users
        mask_uv = (ratings_m[i] != 0) & (ratings_m[j] != 0)
        
        # continue if no intersection
        if np.sum(mask_uv) == 0:
            continue
            
        # get nonzero elements
        ratings_v = ratings_m[i, mask_uv]
        ratings_u = ratings_m[j, mask_uv]
        
        # normalization
        # ...
        
        # for nonzero std
        if len(np.unique(ratings_v)) < 2 or len(np.unique(ratings_u)) < 2:
            continue
        
        similarity_users[i,j] = # your code
        similarity_users[j,i] = # your code

Теперь у нас есть матрицы схожести пользователей. Их можно использовать для рекомендаций.

Для каждого пользователя:

1. Найдём пользователей с похожестью больше $\alpha$ на нашего пользователя.
2. Посчитаем для каждой статьи долю пользователей (среди выделенных на первом шаге), которые взаимодействовали со статьёй.
3. Порекомендуем статьи с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел).

В нашем примере данных не очень много, поэтому возьмём $\alpha = 0$.

После того, как будут сделаны предсказания (новый столбец в interactions), посчитайте качество по той же метрике.

In [ ]:
# your code

interactions['prediction_user_based'] = # your code

### Модель со скрытыми переменными

Реализуем подход с разложением матрицы оценок. Для этого сделайте сингулярное разложение (svd в scipy.linalg), на выходе вы получите три матрицы.

In [ ]:
# your code

Значения у матрицы с сингулярными числами отсортированы по убыванию. Допустим мы хотим оставить только первые 100 компонент (и получить скрытые представления размерности 100). Для этого необходимо оставить 100 столбцов в матрице U, оставить из sigma только первые 100 значений (и сделать из них диагональную матрицу) и 100 столбцов в матрице V. Перемножьте преобразованные матрицы ($\hat{U}, \hat{sigma}, \hat{V^T}$), чтобы получить восстановленную матрицу оценок.

In [ ]:
# your code

Посчитайте качество аппроксимации матрицы по норме Фробениуса (среднеквадратичную ошибку между всеми элементами соответствующими элементами двух матриц). Сравните его с простым бейзлайном с константным значением, равным среднему значению исходной матрицы. У аппроксимации ошибка должна получиться ниже.

In [ ]:
# your code

Теперь можно делать предсказания по матрице. Сделайте их (не забывайте про то, что уже было просмотрено пользователем), оцените качество. Для этого необходимо для каждого пользователя найти предметы с наибольшими оценками в восстановленной матрице.

In [ ]:
# your code

До этих пор мы не проводили никаких преобразований с матрицей оценок. Отцентрируйте все ненулевые (!) значения по каждому пользователю. Сделайте предсказания, посчитайте качество.

In [ ]:
# your code

### Факторизационные машины (опционально)

Попробуем факторизационные машины из библиотеки pyFM (так как можно работать прямо из питона). https://github.com/coreylynch/pyFM

In [ ]:
from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer

Перейдём к обобщению матричных разложений — факторизационным машинам, которые работают могут работать с контентной информацией. Вспомним, какие данные у нас изначально были:

В факторизационную машину можно загрузить "айдишники" пользователей и статей (то есть сделать аналог коллаборативной фильтрации) и одновременно различные признаки.

Удобно обрабатывать категориальные переменные (id и другие) можно с помощью DictVectorizer. Например, процесс может выглядить вот так:

In [ ]:
# train = [
#     {"user": "1", "item": "5", "age": 19},
#     {"user": "2", "item": "43", "age": 33},
#     {"user": "3", "item": "20", "age": 55},
#     {"user": "4", "item": "10", "age": 20},
# ]
# v = DictVectorizer()
# X = v.fit_transform(train)
# y = np.repeat(1.0, X.shape[0])
# fm = pylibfm.FM()
# fm.fit(X,y)
# fm.predict(v.transform({"user": "1", "item": "10", "age": 24}))

Сгенерируйте таблицу с признаками в таком виде, где будут id пользователя, статьи и автора статьи и несколько признаков, которые вы сможете придумать. В качестве целевой переменной возьмите "силу" взаимодействия пользователя с каждой статьёй (помним, что у нас там все примеры по сути положительные). Запустите обучение модели на несколько итераций и сделайте предсказания. Какое качество удаётся достич? 

In [ ]:
train_data = []
test_data = []

for i in tqdm_notebook(range(len(interactions_train_df))):
    features = {}
    features['personId'] = str(interactions_test_df.iloc[i].personId)
    features['contentId'] = str(interactions_test_df.iloc[i].contentId)
    
    # ... 
    
    train_data.append(features)
    
# and test set too

Векторизуем, получим разреженные матрицы.

In [ ]:
dv = DictVectorizer()

train_features = dv.fit_transform(train_data)
test_features = dv.transform(test_data)

In [ ]:
train_features

In [ ]:
fm = pylibfm.FM(num_factors=10, num_iter=30, task='regression')
fm.fit(train_features, y_train)

In [ ]:
# your code

Попробуйте добавить случайные негативные примеры из статей, с которыми пользователь не взаимодействовал. Какое качество удалось достичь?

In [ ]:
# your code

## Часть 3. Контентные  модели

В этой части реализуем альтернативных подход к рекомендательным системам — контентные модели.

Теперь мы будем оперировать не матрицей с оценками, а классической для машинного обучения матрицей объекты-признаки. Каждый объект будет характеризовать пару user-item и содержать признаки, описывающие как пользователя, так и товар. Кроме этого признаки могут описывать и саму пару целиком.

Матрица со всеми взаимодействиями уже получена нами на этапа разбиения выборки на 2 части. 

Придумаем и добавим признаков о пользователях и статьях. Сначала добавим информацию о статьях в данные о взаимодействиях.

In [ ]:
interactions_train_df = interactions_train_df.merge(articles_df, how='left', on='contentId')
interactions_test_df = interactions_test_df.merge(articles_df, how='left', on='contentId')

In [ ]:
# first feature index
features_start = len(interactions_train_df.columns)

После обучения модели нам придётся делать предсказания на тестовой выборке для всех возможных пар статья-пользователь. Подготовим такую матрицу, чтобы параллельно посчитать признаки для неё.

In [ ]:
test_personId = np.repeat(interactions.index, len(articles_df)) 
test_contentId = list(articles_df.contentId) * len(interactions)

test = pd.DataFrame(
    np.array([test_personId, test_contentId]).T,
    columns=['personId', 'contentId'])
test = test.merge(articles_df, how='left', on='contentId')

test.head()

Добавьте признаки-индикаторы возможных значений contentType.

In [ ]:
articles_df.contentType.unique()

In [ ]:
interactions_train_df['is_HTML'] = interactions_train_df.contentType == 'HTML'
# your code

Добавьте признаки "длина названия" и "длина текста".

In [ ]:
# your code

Добавьте признаки-индикаторы языка.

In [ ]:
# your code

Обучим на полученных признаках градиентный бустинг.

In [ ]:
import lightgbm 

regressor = lightgbm.LGBMRegressor()
regressor.fit(interactions_train_df[interactions_train_df.columns[features_start:]],
              interactions_train_df.eventStrength)

Сделайте предсказания на тестовой выборке, сформируйте из них рекомендации. Оцените их качество.

In [ ]:
# your code

Категориальные переменные с большим количеством значений можно закодировать с помощью mean-target кодирования. Закодируйте так id статьи и пользователя. Обучите новую модель и оцените качество.

In [ ]:
# your code

Добавим негативных примеров в выборку — для каждого пользователя выберем случайные статьи, которые он не читал и отметим их нулём. Попробуйте добавить негативных примеров в обучающую выборку.

In [ ]:
# your code